## Creacion de variables y limpieza de datos: ENIF 2021 Profiling
V2 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
ENIF_2021=pd.read_stata('Data/ENIF2021_final.dta')

In [5]:
ENIF_2021Original=ENIF_2021.copy()

### Creacion de variables y limpieza de datos

In [6]:
def SUMA_AMAI(x,y,z,w,k,u):
    suma=0
    #Numero de banos
    if (x==1):
        suma+=24
    elif (x>1):
        suma+=47
    #numero de carros
    if (y==1):
        suma+=22
    elif (y>1):
        suma+=43
    #Internet fijo
    if (z==1):
        suma+=32
    #Numero de cuartos
    if (w==1):
        suma+=15
    elif(w==2):
        suma+=31
    elif(w==3):
        suma+=46
    elif(w>3):
        suma+=61
    #numero de trabajadores
    if(k==1):
        suma+=8
    elif(k==2):
        suma+=16
    elif(k==3):
        suma+=24
    elif(k>3):
        suma+=32
    #Nivel de escolaridad
    if(u==1):
        suma+=6
    elif(u==2):
        suma+=11
    elif(u==3):
        suma+=18
    elif(u==4):
        suma+=23
    elif(u==5):
        suma+=27
    elif(u==6):
        suma+=27
    elif(u==7):
        suma+=27
    elif(u==8):
        suma+=59
    elif(u==9):
        suma+=85
    else:
        suma+=0
    return suma

In [7]:
ENIF_2021['Suma_AMAI'] = ENIF_2021.apply(lambda x: SUMA_AMAI(x['p0_3'],x['p0_4_1a'],x['p0_4_2a'],x['p2_8'],
                                                            x['p0_1'],x['niv']), axis=1)
#ENIF_2021['Suma_AMAI'].isna().sum()

In [8]:
#Niveles socioeconomicos:
# Dict_NSE={"E":1,"D":2,"D+":3,"C-":4,"C":5,"C+":6,"A/B":7}
#NSE_labels=["E","D","D+","C-","C","C+","A/B"]
#print (ENIF_2021['NSE_AMAI'].value_counts())
def NSE_amai(x):
    if x<=47:
        return 1
    elif (x>47) & (x<=94):
        return 2
    elif (x>94) & (x<=115):
        return 3
    elif (x>115) & (x<=140):
        return 4
    elif (x>140) & (x<=167):
        return 5
    elif (x>167) & (x<=201):
        return 6
    else:
        return 7

In [9]:
ENIF_2021['NSE_AMAI'] = ENIF_2021.apply(lambda x: NSE_amai(x['Suma_AMAI']), axis=1)

In [10]:
#Sexo
ENIF_2021['sexo'] = ENIF_2021['sexo'].replace([2],0)
#ENIF_2021['sexo'].value_counts()
## Dict_sexo={"Hombres":1,"Mujeres":2}
#NSE_labels=["Mujeres","Hombres"]

In [11]:

def niv_ingresos(x):
    if (x>0) & (x<=4999):
        return 1
    elif (x>4999) & (x<=13000):
        return 2
    elif (x>13000) :
        return 3
    else:
        return 0

In [12]:
#Nivel de ingresos
#ENIF_2021['ingresos_mensuales'][ENIF_2021['ingresos_mensuales']>0].describe()
#ENIF_2021['ingresos_mensuales'].value_counts()
#ENIF_2021['Nivel_Ingresos'].describe()
### Dict_ingresos={"Sin ingresos reportados":0,"Ingresos menores a 5k":1, "Ingresos entre 5k y 13k":2,
#                    "Ingresos mayores a 13k":3}
#ENIF_2021['Nivel_Ingresos'].value_counts()
ENIF_2021['Nivel_Ingresos'] = ENIF_2021.apply(lambda x: niv_ingresos(x['ingresos_mensuales']), axis=1)

In [13]:
#Tamanio de localidad
#Localidad_labels=["100k> habitantes","15k a 100k habitantes", "2.5k a 15k habitantes","2.5< habitantes"]
#ENIF_2021['tloc'].unique()

In [14]:
#Division por edades
grupos_edad=[
    (ENIF_2021['edad']<35),
    (ENIF_2021['edad']>=35)]

labels_edad=[1,0]
ENIF_2021['Grupos_edad']=np.select(grupos_edad,labels_edad)
#ENIF_2021['Grupos_edad'].value_counts()

In [15]:
ENIF_2021['p3_5'].value_counts()

1    8646
5    2479
8     961
6     683
4     378
3     165
7     156
2      86
Name: p3_5, dtype: int64

In [16]:
ENIF_2021['p3_6'].value_counts()
#Se aplica esta pregunta para los que contestaron en p3_5=[3,4,5,6,8]

6.0    3834
2.0     308
1.0     217
3.0     164
4.0     122
5.0      21
Name: p3_6, dtype: int64

In [17]:
ENIF_2021['p3_7'].value_counts().sum()
#incluye a los que contestaron en la pregunta p3_5=[1,2] y en la pregunta p3_6=[1,2,3,4,5]

9564

In [18]:
#Variable de ocupacion 0's no trabaja, 
#Labels=[1	trabajador(a) sin pago en un negocio familiar o no familiar?
#2	empleada(o) u obrera(o)?
#3	jornalera(o) o peón(a)?
#4	trabajador(a) por su cuenta (no contrata trabajadoras ni trabajadores)?
#5	patrón(a) o empleador(a) (contrata trabajadoras o trabajadores)?]
ENIF_2021['ocupacion']=ENIF_2021['p3_7'].fillna(0)

In [19]:
ENIF_2021['ocupacion'].value_counts()

2.0    5510
0.0    3990
4.0    2618
1.0     683
3.0     581
5.0     172
Name: ocupacion, dtype: int64

In [20]:
def formalidad(x,y):
    if (x>0) & (y>=6):
        return 2
    if (x>0) & (y<6):
        return 1
    else:
        return 0

In [21]:
ENIF_2021['Sector_economia'] = ENIF_2021.apply(lambda x: formalidad(x['ocupacion'],x['p3_10']), axis=1).fillna(0)
#ENIF_2021['Sector_economia'].value_counts()
#Dict_Sector={'Trabajador Informal':2,'Trabajador formal':1, 'No trabaja':0}

In [22]:
cols_demos=['NSE_AMAI','sexo','Nivel_Ingresos','tloc','edad','Grupos_edad','ocupacion','Sector_economia']

In [23]:
columns_headers=list(ENIF_2021.columns.values)

In [24]:
columns_headers

['folio',
 'viv_sel',
 'hogar',
 'n_ren',
 'p3_1_1',
 'p3_1_2',
 'p3_2',
 'p3_3',
 'p3_4',
 'p3_5',
 'p3_6',
 'p3_7',
 'p3_8a',
 'p3_8b',
 'p3_9',
 'p3_10',
 'p3_11',
 'p4_1',
 'p4_2_1',
 'p4_2_2',
 'p4_2_3',
 'p4_2_4',
 'p4_2_5',
 'p4_3',
 'p4_4_1',
 'p4_4_2',
 'p4_4_3',
 'p4_4_4',
 'p4_4_5',
 'p4_4_6',
 'p4_4_7',
 'p4_5',
 'p4_6_1',
 'p4_6_2',
 'p4_6_3',
 'p4_6_4',
 'p4_6_5',
 'p4_6_6',
 'p4_7_1',
 'p4_7_2',
 'p4_7_3',
 'p4_8_1',
 'p4_8_2',
 'p4_8_3',
 'p4_8_4',
 'p4_8_5',
 'p4_8_6',
 'p4_9_1',
 'p4_9_2',
 'p4_9_3',
 'p4_9_4',
 'p4_10',
 'p5_1_1',
 'p5_1_2',
 'p5_1_3',
 'p5_1_4',
 'p5_1_5',
 'p5_1_6',
 'p5_2_1',
 'p5_2_2',
 'p5_2_3',
 'p5_2_4',
 'p5_2_5',
 'p5_2_6',
 'p5_2_7',
 'p5_2_8',
 'p5_2_9',
 'p5_3',
 'p5_4_1',
 'p5_5_1',
 'p5_6_1',
 'p5_7_1',
 'p5_4_2',
 'p5_5_2',
 'p5_6_2',
 'p5_7_2',
 'p5_4_3',
 'p5_5_3',
 'p5_6_3',
 'p5_7_3',
 'p5_4_4',
 'p5_5_4',
 'p5_6_4',
 'p5_7_4',
 'p5_4_5',
 'p5_5_5',
 'p5_6_5',
 'p5_7_5',
 'p5_4_6',
 'p5_5_6',
 'p5_7_6',
 'p5_4_7',
 'p5_5_7',
 'p5_7

In [25]:
#00	Ninguno
#01	Preescolar o kinder
#02	Primaria
#03	Secundaria.
#04	Estudios técnicos con secundaria terminada
#05	Normal básica
#06	Preparatoria o bachillerato
#07	Estudios técnicos con preparatoria terminada.
#08	Licenciatura o ingeniería (profesional)
#09	Maestría o doctorado
ENIF_2021['p3_1_1'].value_counts()

3    3571
2    2977
6    2733
8    2709
0     691
7     315
9     260
4     253
5      35
1      10
Name: p3_1_1, dtype: int64

In [28]:
Datos_ENIF=ENIF_2021.to_csv('Data/ENIF2021_OZMO.csv')

In [45]:
jovenes_BdP=ENIF_2021[(ENIF_2021['edad']<=34)&(ENIF_2021['edad']<=34)].copy()

In [46]:
#Division por edades


grupos_ebdd=[
    (jovenes_BdP['edad']<23),
    ((jovenes_BdP['edad']>=23)&(jovenes_BdP['edad']<27)),
    ((jovenes_BdP['edad']>=27)&(jovenes_BdP['edad']<31)),
     (jovenes_BdP['edad']>=31)   ]

labels_edad=[1,2,3,4]
jovenes_BdP['Grupos_jovbdp']=np.select(grupos_ebdd,labels_edad)

In [50]:
Datos_ENIFJovenes=jovenes_BdP.to_csv('Data/ENIF2021_OZMOJovenes.csv')